In [1]:
# reloading magic
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import seaborn as sns # 0.5.1
sns.set(style="white", palette="muted")
sns.set_context("talk", font_scale=1.4)
%matplotlib qt

In [3]:
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Helvetica'
mpl.rcParams['font.weight'] = 'light'
mpl.rcParams['axes.labelweight'] = 'light'


In [ ]:
# linux
comm_path = os.path.normpath('/home/oliver/repos/bikesim/comm')
scripts_path = os.path.normpath('/home/oliver/repos/bikesim/scripts')

In [4]:
# osx
comm_path = os.path.normpath('/Users/oliver/repos/bikesim/comm')
scripts_path = os.path.normpath('/Users/oliver/repos/bikesim/scripts')

In [5]:
for d in (comm_path, scripts_path):
    if d not in sys.path:
        sys.path.append(d)
from parse_data import *

In [6]:
log_dirname = '/Users/oliver/repos/bikesim/comm/data/exp0'
if log_dirname is None:
    log_dirname = comm_path

In [7]:
subject_map = parse_log_dir(log_dirname)

487 log file(s) parsed for subject(s): 000, 001, 002, 003


In [8]:
fig, ax = plot_historgram(subject_map, 'torque')

In [9]:
torques = []
deltas = []
deltads = []
for subj in subject_map.values():
    t = []
    d = []
    dd = []
    for log in subj.logs:
        t = np.append(t, log.get_field_in_timerange('torque'))
        d = np.append(d, log.get_field_in_timerange('delta'))
        dd = np.append(dd, log.get_field_in_timerange('deltad'))
    # ignore extra entries (time skew should be small)
    num_entries = min(len(t), len(d))
    torques.append(t[:num_entries])
    deltas.append(d[:num_entries])
    deltads.append(dd[:num_entries])

In [10]:
peak = np.where(torques[0] > 10)[0]
counts = []
prev = 0
count = 0
for p in peak:
    if p - prev > 1:
        if count:
            counts.append(count)
        count = 0
    else:
        count += 1
    prev = p
print(counts)
print('avg: {} ms'.format(np.mean(counts) * 20))
print('max: {} ms'.format(max(counts) *  20))


[2, 3, 5, 31, 20, 15, 7, 9, 28, 10, 5, 24, 14, 6, 7, 29, 2, 1, 3, 12, 6, 2, 2, 8, 7, 11, 5, 5, 5, 8, 15, 10, 15, 6, 4, 5, 10, 5, 9, 8, 12, 7, 12]
avg: 190.69767441860463 ms
max: 620 ms


In [11]:
print([len(t) for t in torques])
print([len(dd) for dd in deltads])

[171828, 134004, 160981, 147898]
[171828, 134004, 160981, 147898]


In [12]:
powers = []
for t, dd in zip(torques, deltads):
    powers.append(np.multiply(t, dd))
print('mean power: ', [np.mean(np.abs(p)) for p in powers])
print('std power: ', [np.std(p) for p in powers])

mean power:  [49.821408027638832, 95.565251550813343, 42.830266618245197, 16.63241388012062]
std power:  [674.18583867229358, 1120.5933551427347, 187.63799020245747, 56.424945785153959]


In [13]:
subjects = list(subject_map.values())
print(subjects[0])

In [14]:
for log in subjects[0].logs[:10]:
    y = log.get_field_in_timerange('torque')
    N = len(y)

    yf = np.fft.fft(y)
    xf = np.fft.fftfreq(len(y), 1/50)
    #xf = np.linspace(0.0, 1.0/(2.0*T), N/2)
    print(N/2)
    plt.semilogy(xf[1:N/2], 2.0/N * np.abs(yf[1:N/2]))


82.5
194.0
219.0
316.5
831.5
202.0
169.0
2326.5
1176.0
195.0


/Users/oliver/miniconda3/envs/dev/lib/python3.4/site-packages/ipykernel/__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [15]:
inertia = 0.01 * 100**2 # kg m^2 * (100cm/1m)^2
print(inertia)

100.0


In [16]:
# estimate handlebar inertia
# http://bikethomson.com/mtb-aluminum/
m_bar = 265/1000 # kg
h_bar = 750/1000 # m
r2_bar = 22.8 / 1000 / 2 # m, http://www.sheldonbrown.com/cribsheet-handlebars.html
r1_bar = 18.5 / 1000 / 2 # m, http://www.bikeforums.net/bicycle-mechanics/533030-handlebar-internal-diameter.html

# http://bikethomson.com/elite-x4-31-8-mountain/
m_stem = 141 / 1000 # kg
h_stem = 70 / 1000 # m, 70 mm stem length
r2_stem = 40.64 / 1000 / 2
r1_stem = 31.9 / 1000 / 2

inertia_bar = m_bar/12*(3*(r2_bar**2 - r1_bar**2) + h_bar**2) + m_bar*h_stem**2
inertia_stem = m_bar/12*(3*(r2_stem**2 - r1_stem**2) + h_stem**2) + m_stem*(h_stem/2)**2
print(inertia_bar)
print(inertia_stem)
print(inertia_bar + inertia_stem)

0.013723316334375
0.00029143395170833336
0.014014750286083332


In [17]:
fig, ax = plot_historgram(subject_map, 'deltad')

In [19]:
df = balance_df(subject_map.values())
full_code = df.subject + '.' + df.torque_enabled.astype(int).astype(str)
df['full_subject_code'] = full_code
plt.subplots()
g = sns.boxplot(df.log_timespan, df.full_subject_code.astype(np.float), color="Paired")
g.set_xlabel('subject')

In [101]:
plt.rc('text', usetex=False)
mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'Helvetica'
mpl.rcParams['mathtext.it'] = 'Helvetica:italic'
mpl.rcParams['mathtext.bf'] = 'Helvetica:bold'

#params = {'text.latex.preamble': [r'\usepackage{siunitx}', 
#    r'\usepackage{sfmath}', r'\sisetup{detect-family = true}',
#    r'\usepackage{amsmath}']}   
#plt.rcParams.update(params) 


In [115]:
g1 = plot_dist_grouped_boxchart(subject_map)
#plt.tight_layout()
plt.gca().xaxis.set_ticklabels([])
size = plt.gcf().get_size_inches()
size[1] *= 0.7
size *= 1.3
print(size)
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()

plt.savefig('grouped_boxcharts.pdf', format='pdf', dpi=6000)
#g1[0].suptitle('Distribution of time balanced per subject', fontsize=18)

[ 13.52     6.5065]


In [ ]:
fig2, ax2 = plot_dist_overlapping_histogram(subject_map)
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('overlapping_histogram.pdf', format='pdf', dpi=1000)
# plt.subplots_adjust(top=0.95)
# fig2.suptitle('Distribution of time balanced for all subjects', fontsize=18)

In [ ]:
plt.close('all')
g3 = plot_subject_balance_time_change_boxplot(subject_map)
# plt.subplots_adjust(top=0.9)
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('subject_change_boxplot.pdf', format='pdf', dpi=1000)
# g3.fig.suptitle('Change in time balanced per subject', fontsize=18)

In [ ]:
plt.close('all')
fig4, ax4 = plot_overlapping_psd(subject_map, 'phi', 'longest')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
# plt.savefig('psd_lean_longest.pdf', format='pdf', dpi=1000)
# fig4.suptitle('Power spectral density of lean angle of longest run', fontsize=18)

In [ ]:
plt.close('all')
fig5, ax5 = plot_overlapping_psd(subject_map, 'phi', 'all')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('psd_lean_all.pdf', format='pdf', dpi=1000)
# fig5.suptitle('Power spectral density of lean angle of all runs', fontsize=18)

In [75]:
fig6, ax6 = plot_overlapping_psd(subject_map, 'deltad', 'longest')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
# plt.savefig('psd_steer_rate_longest.pdf', format='pdf', dpi=1000)
# fig6.suptitle('Power spectral density of steer rate of longest run', fontsize=18)

In [116]:
plt.close('all')
fig7, ax7 = plot_overlapping_psd(subject_map, 'deltad', 'all', logy=False, rmslabels=False)
lim = ax7.get_ylim()
ax7.set_ylim((lim[0], 5000))

plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('psd_steer_rate_all.pdf', format='pdf', dpi=1000)
# fig7.suptitle('Power spectral density of steer rate of all run', fontsize=18)

In [50]:
plt.close('all')
fig8, ax8 = plot_overlapping_psd(subject_map, 'delta', 'all')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('psd_steer_all.pdf', format='pdf', dpi=1000)
# fig8.suptitle('Power spectral density of steer of all runs', fontsize=18)

In [48]:
l = subject_map['003'].logs[-1]
print('feedback: {}'.format(l.feedback))
f, axes = plot_lean_steer_yy(l.sensor, l.actuator)
f, axes = plot_subplots(l.sensor, l.actuator, ('phi', 'delta', 'deltad'))

fig, ax = plt.subplots()
tr = l.timerange
ind_a = l.actuator.get_timerange_indices(tr)
ind_s = l.sensor.get_timerange_indices(tr)
f, psds = signal.welch(l.actuator.get_field('phi')[ind_a], 50, nperseg=256, return_onesided=True)
ax.loglog(f, psds, label='lean')
f, psds = signal.welch(l.sensor.get_field('delta')[ind_s], 50, nperseg=256, return_onesided=True)
ax.loglog(f, psds, label='steer')
f, psds = signal.welch(l.sensor.get_field('deltad')[ind_s], 50, nperseg=256, return_onesided=True)
ax.loglog(f, psds, label='steer rate')
ax.set_xlim([f[0], f[-1]])
ax.set_xlabel('frequency [Hz]')
ax.legend()
fig.suptitle('psd')

feedback: False


In [ ]:
for s in subject_map.values():
    t = s.balance_time(feedback=False)
    print(np.mean(t), np.std(t))